<a href="https://colab.research.google.com/github/CeliaDiazSanchez/APRENDIZAJE_AUTOMATICO_23_24/blob/main/Pr%C3%A1ctica_3_Redes_de_Neuronas_Convolucionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3 - Redes de Neuronas Convolucionales
Integrantes: Celia Díaz, Jaime Martínez, Claudia Esnarrizaga


##**Enunciado**
Crea un modelo de Red de Neuronas **Convolucionales** que sea capaz de **reconocer y clasificar imágenes en 100 categorías**. Este modelo será **definido, configurado, entrenado, evaluado y mejorado** para posteriormente usarlo para hacer predicciones.
Para ello tendréis que crear un modelo en **Keras** aplicando de una tirada todos los pasos
al conjunto de datos **CIFAR-100 dataset**, que os podréis descargar de
https://www.cs.toronto.edu/~kriz/cifar.html.
Este conjunto de datos tiene 100 clases que contienen 600 imágenes cada una. Hay 500
imágenes de entrenamiento y 100 imágenes de prueba por clase. Las 100 clases del
CIFAR-100 se agrupan en 20 superclases. Cada imagen viene con una etiqueta "fina" (la clase a la que pertenece) y una etiqueta "gruesa" (la superclase a la que pertenece).

##**Cuestiones a tener en cuenta**

1. Decidir la estructura de la red neuronal convolucional (número de parámetros) y los valores de todos los hiperparámetros que permitan tener la mayor precisión en los datos de test.
2. Asegurar la salida de la llamada *model.summary()* de nuestra red definitiva.
3. Seguir los modelos metodológicos vistos en clase (sobre un modelo base, ir incrementando la complejidad para mejorar el modelo hasta donde se pueda).
4. Ver los conceptos de *batch_normalization* y *dropout* y ver si se podría mejorar el modelo con ello.
5. Ver los conceptos de *callbacks* (Decaimiento del ratio de aprendizaje) y ver si se podría mejorar el modelo con ello.




In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)#

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.14.0


Cargamos nuestros datos

In [2]:
# CARGAMOS EL CONJUNTO DE DATOS FASHION-MINIST EN KERAS
from keras.datasets import  cifar100   # De todos los conjuntos de datos impórtame mnist
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data() # Llama a una función, cargar datos

169001437/169001437 [==============================] - 2s 0us/step


Vamos a ver qué pinta tienen nuestros datos

In [3]:
# Obtener información sobre el conjunto de entrenamiento
print("Shape del conjunto de entrenamiento (imágenes):", train_images.shape)
print("Shape del conjunto de entrenamiento (etiquetas):", train_labels.shape)
print("Número de clases en CIFAR-100:", len(set(train_labels.flatten())))

# Obtener información sobre el conjunto de prueba
print("Shape del conjunto de prueba (imágenes):", test_images.shape)
print("Shape del conjunto de prueba (etiquetas):", test_labels.shape)

Shape del conjunto de entrenamiento (imágenes): (50000, 32, 32, 3)
Shape del conjunto de entrenamiento (etiquetas): (50000, 1)
Número de clases en CIFAR-100: 100
Shape del conjunto de prueba (imágenes): (10000, 32, 32, 3)
Shape del conjunto de prueba (etiquetas): (10000, 1)


Conclusiones que podemos sacar:
- Tamaño del Conjunto de Datos: El conjunto de entrenamiento consta de 50000 imágenes, mientras que el conjunto de prueba tiene 10000 imágenes. Esto es típico en la configuración de conjuntos de datos para el aprendizaje supervisado.
- Resolución de las Imágenes: Cada imagen tiene una resolución de 32x32 píxeles, lo que indica que las imágenes son relativamente pequeñas. Este tamaño más pequeño puede facilitar el entrenamiento rápido de modelos, pero también puede presentar desafíos en la capacidad del modelo para capturar detalles.
- Canales de Color: Las imágenes son imágenes en color con 3 canales (RGB). Cada píxel se representa con tres valores correspondientes a la intensidad de los componentes rojo, verde y azul.
- Número de Clases: El conjunto de datos **CIFAR-100** consta de 100 clases diferentes. Esto significa que cada imagen tiene una etiqueta que pertenece a una de las 100 categorías.
- Dimensiones de las Etiquetas: Las dimensiones del tensor de etiquetas muestran que cada etiqueta es un valor único asociado a cada imagen. La forma **(50000, 1)** indica lo mismo para el conjunto de prueba.
- Preprocesamiento: puede ser necesario realizar algún tipo de preprocesamiento, como normalización de píxeles, para preparar los datos antes de entrenar un modelo.

In [12]:
from keras.utils import to_categorical

train_images = train_images.reshape((50000, 32, 32, 3))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 32, 32, 3))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [4]:
# Obtener las etiquetas numéricas únicas en el conjunto de entrenamiento
unique_labels = set(train_labels.flatten())

# Crear un diccionario de mapeo entre etiquetas numéricas y nombres de clases
label_to_class = {label: f"class_{label}" for label in unique_labels}

print("Diccionario completo de etiquetas y clases:")
print(label_to_class)

Diccionario completo de etiquetas y clases:
{0: 'class_0', 1: 'class_1', 2: 'class_2', 3: 'class_3', 4: 'class_4', 5: 'class_5', 6: 'class_6', 7: 'class_7', 8: 'class_8', 9: 'class_9', 10: 'class_10', 11: 'class_11', 12: 'class_12', 13: 'class_13', 14: 'class_14', 15: 'class_15', 16: 'class_16', 17: 'class_17', 18: 'class_18', 19: 'class_19', 20: 'class_20', 21: 'class_21', 22: 'class_22', 23: 'class_23', 24: 'class_24', 25: 'class_25', 26: 'class_26', 27: 'class_27', 28: 'class_28', 29: 'class_29', 30: 'class_30', 31: 'class_31', 32: 'class_32', 33: 'class_33', 34: 'class_34', 35: 'class_35', 36: 'class_36', 37: 'class_37', 38: 'class_38', 39: 'class_39', 40: 'class_40', 41: 'class_41', 42: 'class_42', 43: 'class_43', 44: 'class_44', 45: 'class_45', 46: 'class_46', 47: 'class_47', 48: 'class_48', 49: 'class_49', 50: 'class_50', 51: 'class_51', 52: 'class_52', 53: 'class_53', 54: 'class_54', 55: 'class_55', 56: 'class_56', 57: 'class_57', 58: 'class_58', 59: 'class_59', 60: 'class_60',

# Cargar el conjunto de datos CIFAR-100
(_, _), (_, cifar100_info) = cifar100.load_data(label_mode='fine')

# Obtener la lista de nombres de las clases (superclases y clases)
class_names = cifar100_info['fine_label_names']

# Crear un diccionario de mapeo entre etiquetas numéricas y nombres de clases
label_to_class = {label: class_name for label, class_name in enumerate(class_names)}

# Crear un diccionario para mapear las superclases a las clases correspondientes
superclass_to_classes = {superclass: [] for superclass in cifar100_info['coarse_label_names']}
for label, class_name in enumerate(class_names):
    superclass = cifar100_info['coarse_label_names'][cifar100_info['coarse_labels'][label]]
    superclass_to_classes[superclass].append(class_name)

# Imprimir las superclases y sus correspondientes clases
for superclass, classes in superclass_to_classes.items():
    print(f"Superclase: {superclass}")
    print("Clases:")
    for class_name in classes:
        print(f"  - {class_name}")
    print("\n")
Da error como código

A continuación, decidiremos la estructura de nuestra red convolucional.
Dado que estamos utilizando imágenes a color, necesitas ajustar el modelo para manejar tres canales en lugar de uno. En tu código, el parámetro input_shape en la capa de entrada debe ser (32, 32, 3).

In [34]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from keras.regularizers import l2

model = Sequential()
model.add(Conv2D(32, (9, 9), activation='relu',kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 24, 24, 32)        7808      
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
Total params: 7808 (30.50 KB)
Trainable params: 7808 (30.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
model.add(Conv2D(64, (9, 9), activation='relu',kernel_regularizer=l2(0.001)))
model.add(MaxPooling2D((2, 2)))

model.summary()

ValueError: ignored

In [36]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

model.add(Flatten())
model.add(Dense(100, activation='softmax'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 24, 24, 32)        7808      
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 12, 12, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 4, 4, 64)          165952    
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 2, 2, 64)          0         
 ng2D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 100)              

In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(train_images, train_labels,
          batch_size=100,
          epochs=5,
          verbose=1)

test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

Epoch 1/5
500/500 [==============================] - 99s 196ms/step - loss: 4.6077 - accuracy: 0.0213
Epoch 2/5
500/500 [==============================] - 95s 191ms/step - loss: 4.3617 - accuracy: 0.0487
Epoch 3/5
365/500 [====================>.........] - ETA: 30s - loss: 4.1655 - accuracy: 0.0759

KeyboardInterrupt: ignored